# Data cleaning

In this notebook I clean the Eurovision voting data from the Eurovision Song Contest Dataset created by [Spijkervet](https://github.com/Spijkervet/eurovision-dataset). 

In [86]:
import pandas as pd
import numpy as np

Load in the data

In [87]:
df = pd.read_csv('data/votes.csv')

To make it easier to work with the data, I add the names of all the countries

In [88]:
# Make it easier to understand by changing the country codes to country names
df['from_country'] = df['from_country'].replace({'at': 'Austria', 'be': 'Belgium', 'bg': 'Bulgaria', 'cy': 'Cyprus', 'cz': 'Czech Republic', 'de': 'Germany', 'dk': 'Denmark', 'ee': 'Estonia', 'gr': 'Greece', 'es': 'Spain', 'fi': 'Finland', 'fr': 'France', 'hr': 'Croatia', 'hu': 'Hungary', 'ie': 'Ireland', 'it': 'Italy', 'lt': 'Lithuania', 'lu': 'Luxembourg', 'lv': 'Latvia', 'mt': 'Malta', 'nl': 'Netherlands', 'pl': 'Poland', 'pt': 'Portugal', 'ro': 'Romania', 'se': 'Sweden', 'si': 'Slovenia', 'sk': 'Slovakia', 'gb': 'United Kingdom', 'ad': 'Andorra', 'al': 'Albania', 'am': 'Armenia', 'au': 'Australia', 'az': 'Azerbaijan', 'ba': 'Bosnia and Herz.', 'by': 'Belarus', 'ch': 'Switzerland', 'cs': 'Serbia', 'ge': 'Georgia', 'is': 'Iceland', 'li': 'Liechtenstein', 'ma': 'Morocco', 'mc': 'Monaco', 'mk': 'North Macedonia', 'md': 'Moldova', 'me': 'Montenegro', 'no': 'Norway', 'rs': 'Serbia', 'ru': 'Russia', 'sm': 'San Marino', 'tr': 'Türkiye', 'ua': 'Ukraine', 'yu': 'Yugoslavia', 'il': 'Israel'})

# And do the same for to_country
df['to_country'] = df['to_country'].replace({'at': 'Austria', 'be': 'Belgium', 'bg': 'Bulgaria', 'cy': 'Cyprus', 'cz': 'Czech Republic', 'de': 'Germany', 'dk': 'Denmark', 'ee': 'Estonia', 'gr': 'Greece', 'es': 'Spain', 'fi': 'Finland', 'fr': 'France', 'hr': 'Croatia', 'hu': 'Hungary', 'ie': 'Ireland', 'it': 'Italy', 'lt': 'Lithuania', 'lu': 'Luxembourg', 'lv': 'Latvia', 'mt': 'Malta', 'nl': 'Netherlands', 'pl': 'Poland', 'pt': 'Portugal', 'ro': 'Romania', 'se': 'Sweden', 'si': 'Slovenia', 'sk': 'Slovakia', 'gb': 'United Kingdom', 'ad': 'Andorra', 'al': 'Albania', 'am': 'Armenia', 'au': 'Australia', 'az': 'Azerbaijan', 'ba': 'Bosnia and Herz.', 'by': 'Belarus', 'ch': 'Switzerland', 'cs': 'Serbia', 'ge': 'Georgia', 'is': 'Iceland', 'li': 'Liechtenstein', 'ma': 'Morocco', 'mc': 'Monaco', 'mk': 'North Macedonia', 'md': 'Moldova', 'me': 'Montenegro', 'no': 'Norway', 'rs': 'Serbia', 'ru': 'Russia', 'sm': 'San Marino', 'tr': 'Türkiye', 'ua': 'Ukraine', 'yu': 'Yugoslavia', 'il': 'Israel'})

I might want to look at voting patterns between neighboring counties so I had chat-GPT write me some convenient code to create a boolean variable telling me if the two countries are neighbors

In [89]:
borders = {
    'Norway': ['Sweden', 'Finland', 'Russia', 'Norway'],
    'Cyprus': [],
    'Denmark': ['Germany', 'Sweden','Norway'],
    'Greece': ['Albania', 'North Macedonia', 'Bulgaria', 'Türkiye'],
    'United Kingdom': ['Ireland'],
    'Finland': ['Sweden', 'Norway', 'Russia'],
    'Spain': ['Portugal', 'France', 'Andorra'],
    'Ireland': ['United Kingdom'],
    'Sweden': ['Norway', 'Finland', 'Denmark'],
    'France': ['Belgium', 'Luxembourg', 'Germany', 'Switzerland', 'Italy', 'Spain', 'Andorra', 'Monaco'],
    'Iceland': [],
    'Portugal': ['Spain'],
    'Belgium': ['Netherlands', 'Germany', 'Luxembourg', 'France'],
    'Switzerland': ['Germany', 'Austria', 'Liechtenstein', 'Italy', 'France'],
    'Netherlands': ['Belgium', 'Germany'],
    'Austria': ['Germany', 'Czech Republic', 'Slovakia', 'Hungary', 'Slovenia', 'Italy', 'Switzerland', 'Liechtenstein'],
    'Germany': ['Denmark', 'Netherlands', 'Belgium', 'Luxembourg', 'France', 'Switzerland', 'Austria', 'Czech Republic', 'Poland'],
    'Moldova': ['Ukraine', 'Romania'],
    'Estonia': ['Russia', 'Latvia'],
    'Israel': ['Lebanon', 'Syria', 'Jordan', 'Palestine'],
    'Luxembourg': ['Belgium', 'Germany', 'France'],
    'Latvia': ['Estonia', 'Russia', 'Belarus', 'Lithuania'],
    'Croatia': ['Slovenia', 'Hungary', 'Serbia', 'Bosnia and Herzegovina', 'Montenegro'],
    'Belarus': ['Russia', 'Latvia', 'Lithuania', 'Poland', 'Ukraine'],
    'Slovenia': ['Italy', 'Austria', 'Hungary', 'Croatia'],
    'Romania': ['Ukraine', 'Moldova', 'Hungary', 'Serbia', 'Bulgaria'],
    'North Macedonia': ['Serbia', 'Kosovo', 'Albania', 'Greece', 'Bulgaria'],
    'Malta': [],
    'Türkiye': ['Greece', 'Bulgaria', 'Georgia', 'Armenia', 'Iran', 'Iraq', 'Syria'],
    'Albania': ['Montenegro', 'Kosovo', 'North Macedonia', 'Greece'],
    'Bosnia and Herzegovina': ['Croatia', 'Serbia', 'Montenegro'],
    'Italy': ['France', 'Switzerland', 'Austria', 'Slovenia', 'Vatican City', 'San Marino'],
    'Lithuania': ['Russia', 'Belarus', 'Latvia', 'Poland'],
    'Ukraine': ['Russia', 'Belarus', 'Poland', 'Slovakia', 'Hungary', 'Romania', 'Moldova'],
    'Serbia': ['Hungary', 'Romania', 'Bulgaria', 'North Macedonia', 'Kosovo', 'Bosnia and Herzegovina', 'Croatia','Montenegro'],
    'Armenia': ['Georgia', 'Azerbaijan', 'Iran', 'Türkiye'],
    'Poland': ['Germany', 'Czech Republic', 'Slovakia', 'Ukraine', 'Belarus', 'Lithuania'],
    'Russia': ['Norway', 'Finland', 'Estonia', 'Latvia', 'Belarus', 'Ukraine', 'Georgia', 'Azerbaijan', 'Kazakhstan', 'China', 'North Korea'],
    'Bulgaria': ['Romania', 'Serbia', 'North Macedonia', 'Greece', 'Türkiye'],
    'Yugoslavia': [],
    'Montenegro': ['Croatia', 'Bosnia and Herzegovina', 'Serbia', 'Albania'],
    'Georgia': ['Russia', 'Armenia', 'Azerbaijan', 'Türkiye'],
    'Azerbaijan': ['Russia', 'Georgia', 'Armenia', 'Iran'],
    'Monaco': ['France'],
    'Hungary': ['Austria', 'Slovakia', 'Ukraine', 'Romania', 'Serbia', 'Croatia', 'Slovenia'],
    'San Marino': ['Italy'],
    'Andorra': ['Spain', 'France'],
    'Czech Republic': ['Germany', 'Poland', 'Slovakia', 'Austria'],
    'Slovakia': ['Poland', 'Czech Republic', 'Austria', 'Hungary', 'Ukraine']}

In [90]:
# Create the neighbors dictionary
neighbors = {}
for key, value in borders.items():
    for neighbor in value:
        neighbors[frozenset([key, neighbor])] = True

# Add a new column to the DataFrame
df['neighboring'] = 0

# Iterate over the rows of the DataFrame
for i, row in df.iterrows():
    if row['to_country'] != '':
        if neighbors.get(frozenset([row['from_country'], row['to_country']])) or neighbors.get(frozenset([row['to_country'], row['from_country']])):
            df.at[i, 'neighboring'] = 1

And then get the region of the to_country and from_country. Here I use [EuroVoc's categorization](https://op.europa.eu/en/web/eu-vocabularies/concept-scheme/-/resource?uri=http://eurovoc.europa.eu/100277) maintained by the European Union. The regions are: Northern Europe, Western Europe, Southern Europe and Central and Eastern Europe. 

In [91]:
# Create the regions dictionary
eurovoc_regions = {
    'Northern Europe': ['Denmark', 'Estonia', 'Finland', 'Iceland', 'Latvia', 'Lithuania', 'Norway', 'Sweden'],
    'Western Europe': ['Andorra', 'Austria', 'Belgium', 'France', 'Germany', 'Ireland', 'Liechtenstein', 'Luxembourg', 'Monaco', 'Netherlands', 'Switzerland', 'United Kingdom'],
    'Southern Europe': ['Cyprus', 'Greece', 'Holy See', 'Italy', 'Malta', 'Portugal', 'San Marino', 'Spain', 'Türkiye'],
    'Central and Eastern Europe': ['Albania', 'Armenia', 'Azerbaijan', 'Belarus', 'Bosnia and Herzegovina', 'Bulgaria', 'Czech Republic', 'Croatia', 'Georgia', 'Hungary', 'Kosovo', 'Moldova', 'Montenegro', 'North Macedonia', 'Poland', 'Romania', 'Russia', 'Serbia', 'Slovenia', 'Slovakia', 'Ukraine', 'Yugoslavia'],
    'Non-European': ['Morocco', 'Israel', 'Australia']
}

# Map each country to its region
df['from_country_region'] = df['from_country'].map({country: region for region, countries in eurovoc_regions.items() for country in countries})

# Map each country to its region
df['to_country_region'] = df['to_country'].map({country: region for region, countries in eurovoc_regions.items() for country in countries})

In [92]:
# Create a new column same_region which is 1 if from_country and to_country are the same region
df['same_region'] = (df['from_country_region'] == df['to_country_region']).astype(int)

#### Standardizing the votes
Throughout the years, the voting system of the Eurovision has changed. To make the votes comparable over time, I standardize all the votes to follow the voting system currently in place. Today the votes are given on a scale from 1-8 and then 10 and 12 - 12 being the highest. 

|Year|Voting system|
|:-|:-|
|1956-1961|10-1|
|1962|3-1|
|1963|5-1|
|1964-1966|5,3,1/6,3/9|
|1967-1970|10-1|
|1971-1973|10-2|
|1974|10-1|
|1975-2015|12, 10 8-1|
|2016-2022|12, 10, 8-1 $x$ 2|


Source: [Wikipedia](https://en.wikipedia.org/wiki/Voting_at_the_Eurovision_Song_Contest)

In [93]:
# Because of the messiness of the data before 1967, we will only look at the data from 1967 onwards
df = df[df['year'] > 1966]

# I can't really standardize the missing 1 value in 1971-1973 but because of the low value, I include those years in the analysis and accept that it might be a bit off

# If the year is higher than 2015 divide the total_points by 2
df['total_points'] = np.where(df['year'] > 2015, df['total_points']/2, df['total_points'])

# If the year is higher than 1974 and the total_points is 10 then change it to 12
df['total_points'] = np.where((df['year'] > 1974) & (df['total_points'] == 10), 12, df['total_points'])

# And if the year is lower than 1975 and the value is 9 then change it to 10
df['total_points'] = np.where((df['year'] < 1975) & (df['total_points'] == 9), 10, df['total_points'])

This standardization is not completely accurate but it will provide a better picture of the strenght between countries over time.

In [94]:
# Get the number of years a country has participated by counting the number of unique years for each to_country
df['years_participated'] = df.groupby('to_country')['year'].transform('nunique')

## Methodological choices
- Only looking at total points scored, not points from jury or public
- Drop rows where `to_country` is equal to `from_country` since countries can't vote for themselves.

In [95]:
df = df[df['from_country'] != df['to_country']]

As both the voting system and the countries that are participating in Eurovision has changed dramatically over the years, I decide to only analyse the votes from the 21st century. If you want to look further back, comment out the next line of code.

In [96]:
# Drop rows where year is below 2000
df = df[df['year'] > 1999]

In [97]:
df.to_csv("data/data_cleaned.csv", index=False)

In [98]:
# Create a new dataframe with each unique country in from_country and year
df2 = df[['from_country', 'year']].drop_duplicates()
df2.to_csv('data/participants.csv')

In [99]:
# Find the number of countries in from_country each year
df['contestants'] = df.groupby('year')['from_country'].transform('nunique')

# Get a dataframe only with the year and contenstants columns
df_contestants = df[['year', 'contestants']].drop_duplicates()
df_contestants.to_csv('data/contestants.csv', index=False)

In [100]:
# Create a list nodes with all the unique countries in from_country and to_country
nodes = list(set(df['from_country'].unique().tolist() + df['to_country'].unique().tolist()))

# Make nodes into a dataframe
nodes = pd.DataFrame(nodes, columns=['country'])

# And save it to a csv file called nodes.csv
nodes.to_csv('data/nodes.csv', index=False)

Some years, the data includes both the tele votes and the jury votes which makes the total votes seem much higher. Also, some years a country makes it to the final making them appear twice in the data. The 'Big Five' (UK, France, Germnay, Italy and Spain) and the host country (the winner of last year) are also given a guaranteed spot in the final, so they only have points from the final. 

To make up for this and make this points more comparable, I group the data by year and countries and get the sum and mean of the points. In some instances this will give some uneven points (with decimals) but I think this is a fair way to make the points comparable.

In [101]:
data = df.groupby(['year', 'from_country', 'to_country','neighboring','from_country_region', 'to_country_region','same_region'])['total_points'].agg(['sum','mean','count']).reset_index()

data.head(2)

,year,from_country,to_country,neighboring,from_country_region,to_country_region,same_region,sum,mean,count
0,2000,Austria,Belgium,0,Western Europe,Western Europe,1,0.0,0.0,1
1,2000,Austria,Croatia,0,Western Europe,Central and Eastern Europe,0,6.0,6.0,1


In [102]:
# Group standardized by 'from_country', 'to_country','neighboring','from_country_region', 'to_country_region','same_region' and calculate the mean of the sum, mean and count columns
all_data = data.groupby(['from_country', 'to_country','neighboring','from_country_region', 'to_country_region','same_region'])['mean'].agg(['mean','count']).reset_index()

# Drop the row if count is less than 5
all_data = all_data[all_data['count'] > 2]

# Drop all the rows where from_country is Yugoslavia
all_data = all_data[all_data['from_country'] != 'Yugoslavia']

# Drop all the rows where to_country is Yugoslavia
all_data = all_data[all_data['to_country'] != 'Yugoslavia']

In [103]:
all_data.to_csv('data/all_data.csv', index=False)

The nodes and edges below can be used to make network graphs of the voting data.

In [104]:
# Create the edges
edges = []
for i, row in all_data.iterrows():
    edges.append((row['from_country'], row['to_country'], row['mean']))

In [105]:
# Make edges a DataFrame
edges_df = pd.DataFrame(edges, columns=['from', 'to', 'value'])

# and save it to a csv file called edges.csv
edges_df.to_csv('data/edges.csv', index=False)

In [106]:
nodes = all_data['from_country'].unique()
nodes_df = pd.DataFrame (nodes, columns = ['countries'])